In [70]:
for var in list(globals()):
    if var != "__name__":
        del globals()[var]
        
import pandas as pd
import numpy as np
import json
import os
from tqdm import tqdm
from global_variables import *

In [82]:
visits_file = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/cbgs_plaec_month_visits_count.csv'
visit_summary_file = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/place_month_visit_summary.csv'
ft_data_path = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/filtered_ft_data.csv'


ft = pd.read_csv(ft_data_path)
visits = pd.read_csv(visits_file,sep="|")
visit_summary = pd.read_csv(visit_summary_file,sep="|")


In [81]:
acs_block_data_file = "C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/acs_block_data.csv.gz"

asc_block_data = pd.read_csv(acs_block_data_file, compression='gzip')

asc_block_data['state'] = asc_block_data['state'].astype(str).str.zfill(2)
asc_block_data['county'] = asc_block_data['county'].astype(str).str.zfill(3)
asc_block_data['tract'] = asc_block_data['tract'].astype(str).str.zfill(6)
asc_block_data['block group'] = asc_block_data['block group'].astype(str)

asc_block_data['CBGS'] = asc_block_data['state'] + asc_block_data['county'] + asc_block_data['tract']+asc_block_data['block group']


asc_block_data['mean_family_income'] = asc_block_data['aggregate_family_income']/asc_block_data['no_of_households']

asc_block_data['white_fraction'] = asc_block_data['population_race_white']/asc_block_data['population_race_base']

asc_block_data['english_speaking_frac'] = asc_block_data['households_language_english']/asc_block_data['households_language_base']

asc_block_data['spanish_speaking_frac'] = asc_block_data['households_language_spanish']/asc_block_data['households_language_base']

asc_block_data['internet_frac'] =asc_block_data['households_internet_yes']/asc_block_data['households_internet_base']

asc_block_data['computer_frac'] =asc_block_data['households_computer_yes']/asc_block_data['households_computer_base']

asc_block_data['under_19_frac'] =asc_block_data['population_by_age_base_under_19']/asc_block_data['population_by_age_base']

asc_block_data['over_65_frac'] = asc_block_data['population_by_age_base_65_older']/asc_block_data['population_by_age_base']

asc_block_data['below_poverty_frac'] = asc_block_data['poverty_status_below']/asc_block_data['poverty_status_base']

asc_block_data['food_stamp_frac'] = asc_block_data['received_food_stamps_snap_yes']/asc_block_data['received_food_stamps_snap_base']

asc_block_data['owner_occ_frac'] = asc_block_data['housing_units_occupancy_own']/asc_block_data['housing_units_occupancy_base']

asc_block_data['home_with_mtg_frac'] = asc_block_data['housing_units_with_a_mortgage']/asc_block_data['housing_units_mortgage_base']


selected_columns = [
    'CBGS',
    'median_age',
    'mean_family_income',
    'white_fraction',
    'travel_time_to_work',
    'spanish_speaking_frac',
    'english_speaking_frac',
    'internet_frac',
    'computer_frac',
    'under_19_frac',
    'over_65_frac',
    'below_poverty_frac',
    'food_stamp_frac',
    'owner_occ_frac'
]
acs = asc_block_data[selected_columns]

In [109]:
unique_cbgs_count = visits['CBGS'].nunique()

# Print the count
print(f"Number of unique CBGS values: {unique_cbgs_count}")

Number of unique CBGS values: 212649


In [114]:
print(visits[visits['CBGS']=='361090013003'][0:10])

                CBGS  no_visits             PLACEKEY         DATE_RANGE_START  \
571863  361090013003          4  222-224@63q-68g-rzf  2019-11-01 00:00:00.000   
869917  361090013003          4  222-223@63q-68g-rzf  2019-11-01 00:00:00.000   

        median_age  mean_family_income  white_fraction  travel_time_to_work  \
571863         NaN                 NaN             NaN                  NaN   
869917         NaN                 NaN             NaN                  NaN   

        spanish_speaking_frac  english_speaking_frac  internet_frac  \
571863                    NaN                    NaN            NaN   
869917                    NaN                    NaN            NaN   

        computer_frac  under_19_frac  over_65_frac  below_poverty_frac  \
571863            NaN            NaN           NaN                 NaN   
869917            NaN            NaN           NaN                 NaN   

        food_stamp_frac  owner_occ_frac  
571863              NaN             NaN

In [117]:
print(ft[ft['PLACEKEY']=='222-224@63q-68g-rzf']['VISITOR_HOME_CBGS'])

25619    {"361090014003":4,"361090013003":4}
Name: VISITOR_HOME_CBGS, dtype: object


In [113]:
cbgs_visits = set(visits['CBGS'].unique())
cbgs_acs = set(acs['CBGS'].unique())

# Find CBGS values in 'visits' that are not in 'acs'
cbgs_not_in_acs = cbgs_visits - cbgs_acs

# Print the number of unique CBGS values not in 'acs'
print(cbgs_not_in_acs)

{'211850307021', '280470014003', '480610116001', '132130102021', '290239501001', '361190017006', '390351179003', '516500103134', '551131005004', '060650456052', '120190303011', '400510007001', '481210201072', '291534701005', 'CA:35370146', '511770201083', '511159513004', '510190304022', '220150111052', '350490012042', '450830224014', '020200023024', '121290102021', '060372409003', '482150210005', '060371134212', '370350102021', '350010014004', '390375550006', '481130056004', '120570110123', '482150221032', '260210112002', '481130107032', '482013339022', '131510704023', '011150405022', '370839309004', '200910537094', '484391101013', '271230406044', '060855019002', '550250017052', '060374641006', '420454068031', '250056302004', '340270403001', '481130039022', '121113821131', '110010073042', '060390005081', '371190056141', '060790129002', '010730142044', '340230082084', '390250415021', '482659606002', '202090411002', '480319501001', '120210111053', '202090404001', '550219704003', '1313505

In [83]:
visits = pd.merge(visits, acs, on='CBGS', how='left')

In [99]:
columns_to_average = [col for col in selected_columns if col != 'CBGS']

def weighted_average(df):
    # total_visits = df['no_visits'].sum()
    weighted_averages = {}
    
    for col in columns_to_average:
        weighted_avg =  np.nansum(df[col] * df['no_visits'])/df.loc[~df[col].isna(), 'no_visits'].sum()
        weighted_averages[f'wa_{col}'] = weighted_avg
    
    return pd.Series(weighted_averages) 

In [100]:
weighted_avg_df = temp.groupby(['PLACEKEY', 'DATE_RANGE_START']).apply(weighted_average)
print(weighted_avg_df)

                                             wa_median_age  \
PLACEKEY            DATE_RANGE_START                         
222-222@3x5-44m-f75 2019-11-01 00:00:00.000           38.9   

                                             wa_mean_family_income  \
PLACEKEY            DATE_RANGE_START                                 
222-222@3x5-44m-f75 2019-11-01 00:00:00.000           62820.153028   

                                             wa_white_fraction  \
PLACEKEY            DATE_RANGE_START                             
222-222@3x5-44m-f75 2019-11-01 00:00:00.000           0.923218   

                                             wa_travel_time_to_work  \
PLACEKEY            DATE_RANGE_START                                  
222-222@3x5-44m-f75 2019-11-01 00:00:00.000                   563.0   

                                             wa_spanish_speaking_frac  \
PLACEKEY            DATE_RANGE_START                                    
222-222@3x5-44m-f75 2019-11-01 00:00:00.000

In [106]:
print(acs[acs['CBGS']=='120210102052'])

               CBGS  median_age  mean_family_income  white_fraction  \
46789  120210102052        59.6       155671.328671        0.846041   

       travel_time_to_work  spanish_speaking_frac  english_speaking_frac  \
46789                  438               0.017812               0.816794   

       internet_frac  computer_frac  under_19_frac  over_65_frac  \
46789        0.92112       0.982188       0.057185      0.307918   

       below_poverty_frac  food_stamp_frac  owner_occ_frac  
46789                 0.0         0.020356         0.92112  


In [94]:
weighted_avg_df = visits.groupby(['PLACEKEY', 'DATE_RANGE_START']).apply(weighted_average)

# Reset the index to make 'place' and 'date' regular columns
weighted_avg_df.reset_index(inplace=True)

KeyboardInterrupt: 

In [103]:
temp = visits[visits['PLACEKEY']=='222-227@8dg-gtw-nkf']

following section cleans ft data into visits and visit_summary

In [ ]:
for var in list(globals()):
    if var != "__name__":
        del globals()[var]
        
import pandas as pd
import json
import os
from tqdm import tqdm
from global_variables import *

In [53]:
# ft_poi_sod_clean.ipynb creates this file
ft_data_path = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/filtered_ft_data.csv'
ft = pd.read_csv(ft_data_path)
ft = ft.dropna(subset=['VISITOR_HOME_CBGS'])
ft = ft.reset_index()

In [66]:
visits_file = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/cbgs_plaec_month_visits_count.csv'
visit_summary_file = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/place_month_visit_summary.csv'
header_option_visits_file = True  
header_option_visit_summary_file = True 

with tqdm(total=len(ft)) as pbar:
    
    for index in range(0, len(ft)):
        
        temp = ft.loc[index]
        
        data = json.loads(temp['VISITOR_HOME_CBGS'])
        df1 = pd.DataFrame(data.items(), columns=['CBGS', 'no_visits'])
        df1['PLACEKEY'] = temp['PLACEKEY']
        df1['DATE_RANGE_START'] = temp['DATE_RANGE_START']
        
        df2 = temp[['PLACEKEY','DATE_RANGE_START','RAW_VISIT_COUNTS','RAW_VISITOR_COUNTS','DISTANCE_FROM_HOME','MEDIAN_DWELL']]
        df2['android'] = 0
        df2['ios'] = 0
        
        try:
            dt = json.loads(temp['DEVICE_TYPE'])
            df2['android'] = dt['android']
        except:
            pass
        
        try:
            dt = json.loads(temp['DEVICE_TYPE'])
            df2['ios'] = dt['ios']
        except:
            pass
        df2_transposed = df2.to_frame().reset_index().T
        df2_transposed.columns = df2_transposed.iloc[0]
        df2_transposed = df2_transposed[1:]
        
        if os.path.isfile(visits_file):
            header_option_visits_file = False  
        if os.path.isfile(visit_summary_file):
            header_option_visit_summary_file = False  
    
        df1.to_csv(visits_file, index=False, mode='a', header=header_option_visits_file,sep="|")
        df2_transposed.to_csv(visit_summary_file, index=False, mode='a', header=header_option_visit_summary_file,sep="|")
        
        pbar.update(1)

100%|██████████| 61521/61521 [05:50<00:00, 175.34it/s]
